## TRABAJANDO CON BASES DE DATOS REMOTAS

### DEFINICIÓN CONTEXTO

In [0]:
# Definición de contexto
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

### CARGANDO DATOS DESDE UN FICHERO CSV

Los parámetros que se usan para cargar el fichero csv que se va a analizar son los siguientes:
- **inferSchema = True**: Este argumento indica a Spark que infiera automáticamente el esquema de datos del archivo CSV. Es decir, Spark intentará determinar los tipos de datos de cada columna basándose en su contenido.
- **header = True**: Este argumento especifica que la primera línea del archivo CSV contiene los nombres de las columnas. Si se establece en True, Spark interpretará la primera fila como nombres de columna.

In [0]:
# Cargando dataset desde un fichero csv, previamente cargado desde el apartado 'Catálogo'--> Crear tabla
heart = spark.read.csv('/FileStore/tables/heart.csv', 
                       inferSchema = True, 
                       header = True)

heart.show()

+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
| 63|  1|  3|     145| 233|  1|      0|    150|    0|    2.3|    0|  0|   1|     1|
| 37|  1|  2|     130| 250|  0|      1|    187|    0|    3.5|    0|  0|   2|     1|
| 41|  0|  1|     130| 204|  0|      0|    172|    0|    1.4|    2|  0|   2|     1|
| 56|  1|  1|     120| 236|  0|      1|    178|    0|    0.8|    2|  0|   2|     1|
| 57|  0|  0|     120| 354|  0|      1|    163|    1|    0.6|    2|  0|   2|     1|
| 57|  1|  0|     140| 192|  0|      1|    148|    0|    0.4|    1|  0|   1|     1|
| 56|  0|  1|     140| 294|  0|      0|    153|    0|    1.3|    1|  0|   2|     1|
| 44|  1|  1|     120| 263|  0|      1|    173|    0|    0.0|    2|  0|   3|     1|
| 52|  1|  2|     172| 199|  1|      1|    162|    0|    0.5|    2|  0|   3|

## CONEXIÓN A UNA BASE DE DATOS EXTERNA

El siguiente código utiliza el comando mágico `%sh` para ejecutar un comando de shell. En el comando tenemos los siguientes parámetros:

- `nc`: Este comando se utiliza para establecer conexiones de red, o para escuchar y enviar datos a través de conexiones de red.
  
- `-vz`: Son opciones del comando `nc`. `-v` le indica a `nc` que muestre información detallada sobre la conexión, y `-z` indica que `nc` debe escanear para ver si el puerto está abierto sin enviar ningún dato.

- `"mysql-jordipozo.alwaysdata.net"`: Es el nombre del host al que se intenta conectar. En este caso, parece ser un servidor MySQL alojado en `mysql-jordipozo.alwaysdata.net`.

- `"3306"`: Es el número de puerto al que se intenta conectar. En el caso de MySQL, el puerto predeterminado es el `3306`.

Este código verifica si es posible establecer una conexión al servidor MySQL alojado en *`mysql-jordipozo.alwaysdata.net`* en el puerto *`3306`* utilizando el comando `nc` para escanear si el puerto está abierto. Esto se usa para verificar la disponibilidad del servidor remoto MySQL desde el entorno de Databricks.

In [0]:
%sh nc -vz "URL" "PUERTO"

Connection to mysql-jordipozo.alwaysdata.net 3306 port [tcp/mysql] succeeded!


#### Establecimiento de conexión
A continuación se establece la conexión a la BB.DD. externa. En este caso se trata de una Base de Datos que usa *MariaDB* y que tenemos alojada en el host *Always Data*

In [0]:
# Conexión a una base de datos
driver = "org.mariadb.jdbc.Driver"

user = 'USUARIO'
password = 'PASSWORD'

database_host = "URL_HOST"
database_port = "3306" # Modificar si el puerto usado no es el que viene por defecto
database_name = "DATABASE_NAME"
table_name = 'TABLE_NAME'

url = f"jdbc:mysql://{database_host}:{database_port}/{database_name}"

print(url)

jdbc:mysql://mysql-jordipozo.alwaysdata.net:3306/jordipozo_midb


#### Recuperando datos de tablas

In [0]:
tblEquipos = (spark.read
    .format("jdbc")
    .option("driver", driver)
    .option("url", url)
    .option("dbtable", table_name)
    .option("user", user)
    .option("password", password)
    .load()
)

#### Accediendo y mostrando información de las tablas

In [0]:
tblEquipos.show()

+---------+------------+-----------+---------+
|   Nombre|      Ciudad|Conferencia| Division|
+---------+------------+-----------+---------+
|    76ers|Philadelphia|       East| Atlantic|
|  Bobcats|   Charlotte|       East|SouthEast|
|    Bucks|   Milwaukee|       East|  Central|
|    Bulls|     Chicago|       East|  Central|
|Cavaliers|   Cleveland|       East|  Central|
|  Celtics|      Boston|       East| Atlantic|
| Clippers| Los Angeles|       West|  Pacific|
|Grizzlies|     Memphis|       West|SouthWest|
|    Hawks|     Atlanta|       East|SouthEast|
|     Heat|       Miami|       East|SouthEast|
|  Hornets| New Orleans|       West|SouthWest|
|     Jazz|        Utah|       West|NorthWest|
|    Kings|  Sacramento|       West|  Pacific|
|   Knicks|    New York|       East| Atlantic|
|   Lakers| Los Angeles|       West|  Pacific|
|    Magic|     Orlando|       East|SouthEast|
|Mavericks|      Dallas|       West|SouthWest|
|     Nets|  New Jersey|       East| Atlantic|
|  Nuggets|  

In [0]:
tblEquipos[['Nombre','Conferencia']].head(4)

Out[36]: [Row(Nombre='76ers', Conferencia='East'),
 Row(Nombre='Bobcats', Conferencia='East'),
 Row(Nombre='Bucks', Conferencia='East'),
 Row(Nombre='Bulls', Conferencia='East')]

In [0]:
tblEquipos[['Nombre','Conferencia']].show(4)

+-------+-----------+
| Nombre|Conferencia|
+-------+-----------+
|  76ers|       East|
|Bobcats|       East|
|  Bucks|       East|
|  Bulls|       East|
+-------+-----------+
only showing top 4 rows



## Lanzando consultas 

In [0]:
df_selected = tblEquipos.select("Nombre", "Ciudad")
df_selected.show()

+---------+------------+
|   Nombre|      Ciudad|
+---------+------------+
|    76ers|Philadelphia|
|  Bobcats|   Charlotte|
|    Bucks|   Milwaukee|
|    Bulls|     Chicago|
|Cavaliers|   Cleveland|
|  Celtics|      Boston|
| Clippers| Los Angeles|
|Grizzlies|     Memphis|
|    Hawks|     Atlanta|
|     Heat|       Miami|
|  Hornets| New Orleans|
|     Jazz|        Utah|
|    Kings|  Sacramento|
|   Knicks|    New York|
|   Lakers| Los Angeles|
|    Magic|     Orlando|
|Mavericks|      Dallas|
|     Nets|  New Jersey|
|  Nuggets|      Denver|
|   Pacers|     Indiana|
+---------+------------+
only showing top 20 rows



### Consultando datos agregados 

- Importamos las funciones de agregado de **pyspark.sql**

In [0]:
from pyspark.sql import functions as F

df_grouped = tblEquipos.groupBy("Conferencia").agg(F.count("Division").alias("Cuenta"))
df_grouped.show()

+-----------+------+
|Conferencia|Cuenta|
+-----------+------+
|       East|    15|
|       West|    15|
+-----------+------+



In [0]:
df_selected.createOrReplaceTempView("temp_view")
df_filtered=spark.sql("SELECT * FROM temp_view WHERE Ciudad = 'Los Angeles'")
df_filtered.show()

+--------+-----------+
|  Nombre|     Ciudad|
+--------+-----------+
|Clippers|Los Angeles|
|  Lakers|Los Angeles|
+--------+-----------+



In [0]:
table_name ="jugadores"
tblJugadores = (spark.read
    .format("jdbc")
    .option("driver", driver)
    .option("url", url)
    .option("dbtable", table_name)
    .option("user", user)
    .option("password", password)
    .load()
)

tblJugadores.show(5)

+------+--------------+-----------+------+----+--------+-------------+
|codigo|        Nombre|Procedencia|Altura|Peso|Posicion|Nombre_equipo|
+------+--------------+-----------+------+----+--------+-------------+
|     1|  Corey Brever|    Florida|   6-9| 185|     F-G| Timberwolves|
|     2|  Greg Buckner|    Clemson|   6-4| 210|     G-F| Timberwolves|
|     3|Michael Doleac|       Utah|  6-11| 262|       C| Timberwolves|
|     4|    Randy Foye|  Villanova|   6-4| 213|       G| Timberwolves|
|     5|    Ryan Gomes| Providence|   6-7| 250|       F| Timberwolves|
+------+--------------+-----------+------+----+--------+-------------+
only showing top 5 rows



### Realizando consultas sobre varias tablas (Usando JOIN y WHERE)

In [0]:
tblEquipos.createOrReplaceTempView("equipos")
tblJugadores.createOrReplaceTempView("jugadores")

In [0]:
df_join = spark.sql("select e.Nombre, e.Conferencia, j.Nombre from equipos e join jugadores j on e.Nombre == j.Nombre_equipo where e.Nombre='76ers'")
df_join.show()

+------+-----------+----------------+
|Nombre|Conferencia|          Nombre|
+------+-----------+----------------+
| 76ers|       East|  Louis Amundson|
| 76ers|       East|    Calvin Booth|
| 76ers|       East|   Rodney Carney|
| 76ers|       East|Samuel Dalembert|
| 76ers|       East|    Reggie Evans|
| 76ers|       East|    Willie Green|
| 76ers|       East|    Herbert Hill|
| 76ers|       East|  Andre Iguodala|
| 76ers|       East|    Andre Miller|
| 76ers|       East|     Kevin Ollie|
| 76ers|       East|Shavlik Randolph|
| 76ers|       East|     Jason Smith|
| 76ers|       East|  Louis Williams|
| 76ers|       East|  Thaddeus Young|
+------+-----------+----------------+



### EJERCICIOS PRÁCTICOS

1. ¿Cuál es el jugador que más kilos pesa? 
2. ¿Cuánto mide el jugador más bajo y cuál es su nombre?  
3. ¿Cuántos jugadores tiene el equipo de los Lakers? 
4. ¿Cuánto pesan de media los jugadores de los Maveriks?